In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
kc1 = pd.read_csv(r'/content/kc1_csv.csv')
print('kc1 shape: ', kc1.shape)
kc1.head()

kc1 shape:  (2109, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,...,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,...,37,2,5,0,16.0,28.0,89.0,52.0,15.0,True
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,...,21,0,2,0,11.0,10.0,41.0,17.0,5.0,True


In [3]:
kc1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                2109 non-null   float64
 1   v(g)               2109 non-null   float64
 2   ev(g)              2109 non-null   float64
 3   iv(g)              2109 non-null   float64
 4   n                  2109 non-null   float64
 5   v                  2109 non-null   float64
 6   l                  2109 non-null   float64
 7   d                  2109 non-null   float64
 8   i                  2109 non-null   float64
 9   e                  2109 non-null   float64
 10  b                  2109 non-null   float64
 11  t                  2109 non-null   float64
 12  lOCode             2109 non-null   int64  
 13  lOComment          2109 non-null   int64  
 14  lOBlank            2109 non-null   int64  
 15  locCodeAndComment  2109 non-null   int64  
 16  uniq_Op            2109 

In [4]:
target = 'defects'
y = kc1[target]
x=kc1.drop(['defects'], axis=1)
x.shape

(2109, 21)

In [5]:
#x.corr()
#x.corr(method='pearson',min_periods=0)
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
             #if we want to conseder -ve corelation to be treated as +ve just add abs just after if in above statement like:-
             # if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [6]:
corr_features = correlation(x, 0.8) #can change treshold value accordinglly with current values i will be removing 18 highly corelated features
len(set(corr_features))
selector=corr_features
len(set(corr_features))

17

In [7]:
x.shape

(2109, 21)

In [8]:
x.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.00,1,1,1,1,1.0,1.0,1.0,1.0,1.0
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,...,1187.70,65,10,6,0,18.0,25.0,107.0,64.0,21.0
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,...,635.37,37,2,5,0,16.0,28.0,89.0,52.0,15.0
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,...,132.33,21,0,2,0,11.0,10.0,41.0,17.0,5.0


In [9]:
xn=x.drop(selector,axis=1)

In [10]:
xn.head()

,loc,l,lOComment,locCodeAndComment
0,1.1,1.30,2,2
1,1.0,1.00,1,1
2,83.0,0.04,10,0
3,46.0,0.07,2,0
4,25.0,0.11,0,0


In [11]:
xn.shape

(2109, 4)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(xn, y, test_size=0.2, random_state=42, shuffle=True)

In [13]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2109, 21)
(2109,)
-------------------
(1687, 4)
(1687,)
(422, 4)
(422,)


In [14]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8293838862559242
Th confusion matrix of SVM:
  [[347   9]
 [ 63   3]]


In [15]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[347   9]
 [ 63   3]]
              precision    recall  f1-score   support

       False       0.85      0.97      0.91       356
        True       0.25      0.05      0.08        66

    accuracy                           0.83       422
   macro avg       0.55      0.51      0.49       422
weighted avg       0.75      0.83      0.78       422



In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8222748815165877
The confusion matrix of KNN:
  [[341  15]
 [ 60   6]]
[[341  15]
 [ 60   6]]
              precision    recall  f1-score   support

       False       0.85      0.96      0.90       356
        True       0.29      0.09      0.14        66

    accuracy                           0.82       422
   macro avg       0.57      0.52      0.52       422
weighted avg       0.76      0.82      0.78       422



In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.8009478672985783
[[318  38]
 [ 46  20]]
              precision    recall  f1-score   support

       False       0.87      0.89      0.88       356
        True       0.34      0.30      0.32        66

    accuracy                           0.80       422
   macro avg       0.61      0.60      0.60       422
weighted avg       0.79      0.80      0.80       422



In [18]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8322465915826912

In [19]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.88      0.93      0.90       356
        True       0.43      0.29      0.35        66

    accuracy                           0.83       422
   macro avg       0.65      0.61      0.62       422
weighted avg       0.81      0.83      0.81       422

[[331  25]
 [ 47  19]]
